# Author : Melissa Onwuka 

In [2]:
pip install tensorflow 

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install OS 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement OS (from versions: none)
ERROR: No matching distribution found for OS


In [4]:
import numpy as np 
import pandas as pd

In [5]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [7]:
import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [8]:
import os
os.getcwd()

'C:\\Users\\onwuk'

In [10]:
mel = pd.read_csv("movies_to_emojis.csv")

In [12]:
mel.head()

,Movies,Emojis
0,Titanic,⛴️🌊❄️💔👫
1,The Lion King,🦁👑🎶
2,Avatar,🌍🌳💙👽
3,The Dark Knight,🌑🦇🌆🃏
4,The Shawshank Redemption,🗝️🤝👨⚖️


In [14]:
print("Total number of movies:", len(mel))
print("Total number of unique emojis:", mel["Emojis"].nunique())

Total number of movies: 253
Total number of unique emojis: 251


In [16]:
train_data, test_data = train_test_split(mel, test_size=0.2, random_state=42)

In [23]:
train_sequences = tokenizer.texts_to_sequences(train_data["Movies"])
test_sequences = tokenizer.texts_to_sequences(test_data["Movies"])

In [17]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data["Movies"])
vocab_size = len(tokenizer.word_index) + 1

In [24]:
max_seq_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in test_sequences))
train_padded = pad_sequences(train_sequences, maxlen=max_seq_length, padding="post")
test_padded = pad_sequences(test_sequences, maxlen=max_seq_length, padding="post")

In [25]:
max_seq_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in test_sequences))
train_padded = pad_sequences(train_sequences, maxlen=max_seq_length, padding="post")
test_padded = pad_sequences(test_sequences, maxlen=max_seq_length, padding="post")

In [27]:
emoji_labels = mel["Emojis"].unique()
emoji_label_indices = {label: i for i, label in enumerate(emoji_labels)}
train_labels = train_data["Emojis"].map(emoji_label_indices)
test_labels = test_data["Emojis"].map(emoji_label_indices)

In [28]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_seq_length))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(64))
model.add(Dense(len(emoji_labels), activation="softmax"))

In [29]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
model.fit(train_padded, train_labels, epochs=10, validation_data=(test_padded, test_labels))

Epoch 1/10
7/7 [==============================] - 6s 196ms/step - loss: 5.5275 - accuracy: 0.0000e+00 - val_loss: 5.5323 - val_accuracy: 0.0000e+00
Epoch 2/10
7/7 [==============================] - 0s 14ms/step - loss: 5.5209 - accuracy: 0.0347 - val_loss: 5.5421 - val_accuracy: 0.0000e+00
Epoch 3/10
7/7 [==============================] - 0s 18ms/step - loss: 5.5153 - accuracy: 0.0446 - val_loss: 5.5566 - val_accuracy: 0.0000e+00
Epoch 4/10
7/7 [==============================] - 0s 18ms/step - loss: 5.5076 - accuracy: 0.0644 - val_loss: 5.5833 - val_accuracy: 0.0000e+00
Epoch 5/10
7/7 [==============================] - 0s 21ms/step - loss: 5.4941 - accuracy: 0.0198 - val_loss: 5.6566 - val_accuracy: 0.0000e+00
Epoch 6/10
7/7 [==============================] - 0s 15ms/step - loss: 5.4633 - accuracy: 0.0099 - val_loss: 5.9451 - val_accuracy: 0.0000e+00
Epoch 7/10
7/7 [==============================] - 0s 20ms/step - loss: 5.3849 - accuracy: 0.0050 - val_loss: 6.9924 - val_accuracy: 0.000

In [31]:
new_movie_names = ["Avengers", "A Silent Voice"]
new_sequences = tokenizer.texts_to_sequences(new_movie_names)
new_padded = pad_sequences(new_sequences, maxlen=max_seq_length, padding="post")
predictions = model.predict(new_padded)

1/1 [==============================] - 1s 926ms/step


In [32]:
predicted_emojis = [emoji_labels[prediction.argmax()] for prediction in predictions]

In [33]:
for movie, emoji in zip(new_movie_names, predicted_emojis):
    print(f"Movie: {movie}   Emoji: {emoji}")

Movie: Avengers   Emoji: 🚀🌌🧑🚀
Movie: A Silent Voice   Emoji: 🕶️🏍️🌄
